# QMUL Autodesk AI Lab Data Science Challenge 2019


<img src="./images/StyleExamples.png" width="500"><img> 

Autodesk AI Lab challenge entails analysing a couple of datasets of 3D shapes, represented as [triangle meshes](https://en.wikipedia.org/wiki/Triangle_mesh) and then providing the designer insights about the style of a shape by visualising similarities between different regions of the shapes. You will be provided with open source datasets, python packages for reading and visualising the meshes and some helper code to kick-start with. Experience with Python and Jupyter notebooks is very helpful in this challenge.

You will need, at least, a computer with access to internet, Google Chrome browser installed, and a google drive account to use Google Colab for the compute. This is our recommended way to work on this challenge. To open a copy of this jupyter notebook inside Colab click on the following badge. 
You can find an intro to Google Colab [here](https://colab.research.google.com/notebooks/welcome.ipynb).
Also, there are some good quick tips and examples for using Colab [here](https://rozbeh.github.io/colab_101.html). 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hoosha/adskdschlng2019/blob/master/adskdschlng2019.ipynb)

Obviously, you can also use your own laptop if you have jupyter notebook installed and enough disk space and compute power on it. We recommend pip install for Linux, and conda for MacOS and Windows.

This jupyter notebook walks you through preparation steps for the challenge day and also explains the main challenge. We recommend you to follow through it all before the challenge day if you feel interested in this challenge.


## Triangle Meshes 
Our data consist of [Triangle Meshes](https://en.wikipedia.org/wiki/Triangle_mesh) (or tri-mesh for short) that represent the shape of 3D objects. If you remember your 3D Computer Graphics course, these figures here must be enough to refresh your memory. If you are not familiar with them, spend some time to [learn](https://github.com/alecjacobson/computer-graphics-csc418/blob/32fac2f906b931571911bffa5b81cda87c69406a/lectures/lecture5.pdf) [more](https://www.youtube.com/watch?v=s4lYj40_tnY) [about](http://graphics.stanford.edu/courses/cs248-18-spring-content/lectures/07_geometryprocessing/07_geometryprocessing_slides.pdf) [them](http://courses.cms.caltech.edu/cs171/assignments/hw5/hw5-notes/notes-hw5.html).

<table><tr>
    <td><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Mesh_overview.svg/1440px-Mesh_overview.svg.png" width="700"></td>
    <td><img src="http://www.lix.polytechnique.fr/~maks/Verona_MPAM/TD/TD2/images/snapshot01.png" width="200"></td>
</tr></table>

<table>
    <td><img src="https://www.scratchapixel.com/images/upload/shading-intro/shad-tri-normal.png?" alt="Face normal" width=200/></td>
    <td><img src="http://courses.cms.caltech.edu/cs171/assignments/hw5/hw5-imgs/normals.png" alt="normals" width="200"> </img></td>
</tr></table>

___

## Datasets

For this challenge you have access to two open-source datasets (you don't need to download these):
1. [**Thingi10k dataset**](https://ten-thousand-models.appspot.com/): We use a re-meshed version of this dataset (that is surface-meshed by [TetWild](https://github.com/Yixin-Hu/TetWild)). You can download this from [here](https://drive.google.com/file/d/1E_C1uVoG1ZGF3pfDpHFKIS8Qqd2VXLZQ/view). This is an unlabelled dataset of mostly digital 3D shapes designed directly by people in CAD software mostly to be used for 3D printing. You can read more about it [here](https://www.groundai.com/project/thingi10k-a-dataset-of-10000-3d-printing-models/2) but keep in mind that we are not using the original dataset but the version that is re-meshed by [TetWild](https://github.com/Yixin-Hu/TetWild) with surface mesh extracted. So, some of the meshes might have less problems compared to what is described there and some meshes might have other problems that is not mentioned there! This dataset provides less noisy data and a good aesthetic and stylistic diversity.
![](https://storage.googleapis.com/groundai-web-prod/media%2Fusers%2Fuser_14%2Fproject_86411%2Fimages%2Fx1.png)


2. A dataset of **Chess Pieces**: from this [paper](https://tinyurl.com/y2yr3za8) that you can download from [here](http://www2.compute.dtu.dk/~janba/scansdp.tgz). Thanks to [Jakob Andreas Bærentzen](https://tinyurl.com/yxk6fbkt) and the other authors for sharing the dataset. This is a very small labelled dataset. The filenames specify the type of the chess piece and the set they belong to. The meshes are generated by scanning actual chess pieces and therefor the meshes are of higher resolution but a bit noisier compared to the Thingi10k dataset. This dataset provides a good sense of style and aesthetics and labels about the content and style of the meshes. You can get good inspiration about the direction of your work although the distribution of this dataset might be slightly different from the Thingi10k dataset.

As you expect, both datasets contain some corrupted, noisy, or other problematic data and require cleaning and data wrangling. These datasets contain tri-meshes in .obj file format. In the next step we download these datasets into the Colab environment and decompress them.

### Download and decompress datasets
To download the Thingi10k dataset run the following.

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1E_C1uVoG1ZGF3pfDpHFKIS8Qqd2VXLZQ',
                                    dest_path='./downloads/10k_surface.tar.gz')

And to unzip it...

In [ ]:
!tar -xvf downloads/10k_surface.tar.gz 

After this step, your mesh files must be in ./10k_surface/

To Download and unzip the chess dataset run the following steps:

In [ ]:
!wget -nv "http://www2.compute.dtu.dk/~janba/scansdp.tgz" -P downloads

In [ ]:
!tar -xvf downloads/scansdp.tgz

The chess piece .obj files are now in ./scansdp/

### Loading data
We use [Trimesh](https://github.com/mikedh/trimesh) Python package to load/save/manipulate/visualise tri-meshes. It is also very handy for gathering the data from the vertices, edges and faces of a mesh. You can find its documentation [here](https://trimsh.org/) and a [Gitter forum](https://gitter.im/trimsh/Lobby) to ask your questions (the [author](https://github.com/mikedh) is in GMT-4 time zone). The following command installs Trimesh on your Colab machine, if it isn't already installed.

In [ ]:
!pip freeze | grep -qF 'trimesh==' || pip install trimesh[all]

In [ ]:
import trimesh

To load a mesh file in your Colab machine you can simply use the following:

In [ ]:
mesh1 = trimesh.load_mesh('scansdp/dp007K.obj')
mesh2 = trimesh.load_mesh('scansdp/dp007B.obj')

You can access the vertices (an array of vertex positions in 3D space) as follows:

In [ ]:
mesh1.vertices

You can access the faces (an array of 3 vertex indices that make the triangles) as follows:

In [ ]:
mesh1.faces

The vertex indices (that by the way, start from 0 referring to the first row in the mesh.vertices array) are listed in counter-clockwise order, as shown in the following figure. This order specifies the direction of the face normal (unit vectors perpendicular to each triangle) based on the right-hand rule. That means in this figure the normal vectors are pointing out of the screen towards you looking at the mesh from outside.

<img src="./images/TrimeshData.png" alt="normals" width="500"><img>

You can get the normal vectors corresponding to the faces as follows:

In [ ]:
mesh1.face_normals

You can also access the edges (arrays of 2 vertex indices making the edges)...

In [ ]:
mesh1.edges_unique

And the adjacent faces (indices of two faces on both sides of the corresponding edges)...

In [ ]:
mesh1.face_adjacency

Look at the shapes of these data structures and you can find the total number of vertices, faces, and edges:

In [ ]:
mesh1.vertices.shape, mesh1.faces.shape, mesh1.edges_unique.shape, mesh1.face_adjacency.shape

### Visualising data

Then, visualising the mesh inside the jupyter notebook is as simple as the following. Just resize your browser to fix the aspect ratio and see the object correctly if it is not shown properly. Use mouse wheel (scroll) to zoom in and out and left-click-frag to rotate, and right-click-drag to pan.

In [ ]:
mesh1.show(smooth=False)

It is easy to move a mesh around, colourise the faces, make a single mesh from two meshes, create a scene from them, and then show the scene:

In [ ]:
# Move mesh1 40 units in x direction
mesh1.apply_translation([40,0,0])

# Color the faces of mesh2 using the 'jet' color map and using the value
# of the z position of the first vertex of each face
mesh2.visual.face_colors = trimesh.visual.interpolate(mesh2.vertices[mesh2.faces[:,0],2],color_map='jet')

# Make a single mesh out of them and then create a scene from them.
sc=trimesh.scene.Scene(mesh1 + mesh2)

# Show the scene
sc.show(smooth=False)

Look at [trimesh quick start guide](https://github.com/mikedh/trimesh/blob/master/README.md#quick-start) to learn more about some of the other things you can quickly do with it. 

### Other Tools
There are also other tools that you can run locally on your laptop in case you need to manipulate or inspect triangular meshes. These tools usually take some time to learn. So, if you haven't used them before, take your time before the challenge day to familiarise yourself with the first one. They come very handy in case you need to check a mesh topology, a face/vertex index or eye ball something on a mesh to find out quickly what is going wrong.

#### MeshMixer
[Autodesk MeshMixer](http://www.meshmixer.com) is a free but very powerful triangle mesh editing tool (Mac and Windows) that can be used in 3D Printing, Game Development, mesh manipulation, inspection, and repair among other things.

#### Paraview
[Paraview](https://www.paraview.org) is an open-source, multi-platform data analysis and visualization application from [Kitware](https://www.kitware.com) that can be used to visualize and inspect triangular meshes among many other forms of 3D data.


#### TetWild
You can use [TetWild](https://github.com/Yixin-Hu/TetWild) (the tool that generated the re-meshed version of the dataset that we are using here) to re-mesh any mesh locally on your machine. In case you were not happy with the way it was meshed or wanted to mesh the dataset with different TetWild parameters, just look at its github repository and install it locally on your laptop. Re-meshing operations take some time and we don't recommend to re-mesh the whole dataset. Maybe the quickest way to install and use it on your machine is to run it in a docker container on your laptop, as explained in TetWild repository readme page. Two main parameters are epsilon and edge_length. Also, keep in mind that we only need the surface mesh (tri-mesh) and not the tetrahedral mesh (tet-mesh).

Spend your time before the challenge day to familiarise yourself with the tools, datasets and their deficiencies, and also the main challenge objectives as follows.

___

## Main Challenge
The main point of the challenge is to provide designers with useful insights about stylistic and aesthetic features of a shape in the context of another shape or set of shapes. Assume we have one (or a set of) meshes with a certain desired style and aesthetics (like a set of chess pieces that go together). Now assume a designer is working on the design of a new chess piece and likes to know if parts of her design don’t match the style and aesthetics of the rest of the chess set. The insights can be, for example, in the form of visualisation of regions of a mesh that are not similar to any region of another mesh (or set of meshes) with red colour (representing undesired local forms and features), and regions that are very similar to some regions of the other mesh(s) in blue. The designer can quickly know which part of her design require editing.

For example, look at these two shapes (not from the datasets):

<img src="./images/Parts.png" alt="Two example shapes" width="600"></img>


In the above figure, if we compare the larger complex shape (on the top-right) with the smaller simpler shape (on the bottom-left), we see that almost all the local regions on the larger shape are more or less similar to at least one region in smaller shape. Except for the fillets on the bottom edges of the pockets that have no counterparts in the smaller shape. This is shown with red regions (high dissimilarity) vs dark blue regions (low dissimilarity) in the example visualization below. A designer looking at this visualization can tell right away, which parts of her design do not match with the style of the smaller shape. This is the type of visualization we expect in this challenge.

<img src="./images/FinalVisExample.png" alt="Visualization Example" width="600"></img>


The core of the problem is to engineer suitable features (or learn it from data) that describe properties of a small mesh patch (say, a few adjacent triangles) that allows us to measure the stylistic similarity of that region compared to one or more other meshes. 

The challenge consists of four main steps and a bonus step:
1. Data wrangling 
2. Latent representation
3. Similarity metric
4. Visualization
5. Bonus step

---

**Step 1 - Data wrangling:** Define a large-enough mesh patch as the data atom and then gathering data about these mesh patches from each mesh. In its simplest form you can think of one triangle as the patch and use its properties and its relations with its adjacent triangles as the data atom. Selecting the right properties and relations that are less dependent on the meshing and the size and global shape of the mesh and more dependent on the local style and aesthetic of that region of the mesh will help you filter out the irrelevant data and capture the signal you are interested in.


If you feel that you need to normalize the meshes (make sure they are roughly of the same size) you can use the following function to normalize them. This function moves the mesh to the origin (object centroid at 0,0,0) and scales it up or down so that its oriented bounding box (oriented with the coordinate system)

In [ ]:
def normalize_mesh_(mesh):
    '''
    Normalizes a mesh (in place) and returns centroid and scaling factor
    :param mesh: The main mesh used to calculate the center and scaling 
    factor. Will be normalized in place.
    
    :return c: the centroid coordinates of the mesh that could be used to 
            re-translate the mesh back to original position.
            s: the the scaling factor of the mesh to be used for re-scaling
            back to original size.
    '''
    c = mesh.bounding_box_oriented.centroid
    s = 2 / mesh.bounding_box_oriented.extents.max()
    mesh.apply_translation(-c)
    mesh.apply_scale(s)
    return c, s

Also, as you analyse the meshes looking for the signals relevant to the local style of the shapes you will find that the style of the shapes and mesh patches don't change if you look at them from different angles, that is to rotate them along the vertical (z) axis. You are not required to take into account this invariance, although you can. However, if you turn them upside down (rotate them along x and/or y axis), the style of the local patches will be affected. For example, think about a set of chess pieces that all have some sort of cylindrical style (as shown in the following figure). If you rotate one of the pieces 90 degree along x axis, the patch styles of this piece will change and instead match with a different chess set that has flat regions in the front and back and curves on the top, bottom, and the sides. So, you can use this knowledge about the effect of orientation, particularly the sense of up/down for the shapes, to your advantage. Basically, we recommend not to rotate the meshes and compare their patches in their original orientations from the datasets.

<img src="./images/match.png" alt="normals" width="600"><img>

---

**Step 2 - Latent representation:** Perform some kind of [Feature Engineering]( https://en.wikipedia.org/wiki/Feature_engineering) or [Representation Learning]( https://en.wikipedia.org/wiki/Feature_learning) or any other approach to generate a latent vector (embedding) from each patch. You need to focus on the variance in the data that comes from stylistics and aesthetic properties of a mesh patch (signal) versus other variances due to general shape and size of the object and the mesh (noise). The labels in the Thingi10k dataset might be helpful to inspire you or provide useful signals for this step. Be careful though. Unfortunately, the chess dataset is from a 3D scanner (not CAD) and a bit noisy. That’s data scientists’ life! You might want to use a deep neural network or other ML techniques to generate a latent vector (patch style embedding) from each patch that somehow represent the stylistic and aesthetic properties of that patch or just a dimensionality reduction or even use the feature vector directly if you believe it has a good style signal and very low noise and irrelevant variance. At the end of this stage you must have a piece of code that outputs that style embedding for an input patch data (coming from any one of the meshes in the Thingi10k dataset).

---

**Step 3 – Similarity metric:** Come up with a good similarity metric (a distance or divergence) in order to compare patches with each other. Assuming the style embedding you generate in the previous step is consistent and representative of the local style of the patch, you can measure the distance between two embeddings. This distance will give us a sense of dissimilarity between the local style of the patches.

---
**Step 4 – Visualisation:** You can use the similarity metric between every patch in mesh A with every patch in mesh B and then take, for example minimum value (over all patches in B) or some other statistics to find a distance of every patch in A with the style of B. Then you can visualise this by using this value to colourise the mesh A faces based on their patch similarity to style of B. 

---
**Bonus step:** You can use a similar approach for visualising the style similarity of the patches in a mesh A to a set of meshes S={B,C,D,…} (that are all similar to each other in style), effectively visualising the similarity of the style of different patches of A with style of S.